In [1]:
%load_ext autoreload
%autoreload 2

from model_utils import train_model, split_data, split_branches
from config import Config
import json
import matplotlib.pyplot as plt
%matplotlib inline
import math
import random
import os
import datasets
import torch
import transformers
from model_manager import ModelManager

In [2]:
seed = random.randint(0, 2 ** 32 - 1)
random.seed(seed)
datasets.logging.set_verbosity(datasets.logging.ERROR)
# Tell pytorch to run this model on the GPU.
device_name = "cuda:0" if torch.cuda.is_available() else "cpu"
# device_name = "cpu"
device = torch.device(device_name)
print(f"Will use {device_name} for training with seed: {seed}")

Will use cuda:0 for training with seed: 1880371636


In [3]:
split_data(os.path.join(Config.work_dir, "awsw_story_input.txt"))

In [4]:
config = {
    "model_name": "EleutherAI/gpt-neo-125M",
    "lr": 6e-4,
    "warmup_factor": 0,
    "scheduler": "polynomial_decay_schedule_with_warmup",
    "lr_end": 2e-6,
    "power": 0.6,
    #"freeze_layer_rate": 1e-4,
    "freeze_from_steps": -1,
    "seed": seed,
    "num_epoch": 50,
    "to_freeze_count": 150
}

optuna_result_attachement = {
    'lr': 0.001,
    'scheduler': 'cosine_schedule_with_warmup',
    'to_freeze_count': 150,
    'warmup_factor': 1
}
config.update(optuna_result_attachement)
config

{'model_name': 'EleutherAI/gpt-neo-125M',
 'lr': 0.001,
 'warmup_factor': 1,
 'scheduler': 'cosine_schedule_with_warmup',
 'lr_end': 2e-06,
 'power': 0.6,
 'freeze_from_steps': -1,
 'seed': 1880371636,
 'num_epoch': 50,
 'to_freeze_count': 150}

In [ ]:
train_results = {}
train_model(config, train_results, device = device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Dataset demo snapshot:
<p><msg>c "I see."<d><scn>town4<msg>Ad "Are you ready to see some stunt flying?"<|endoftext|><d><scn>park2<msg>Ry "I see."<|endoftext|><p><msg>c "Alright."<|endoftext|><p><msg>c "So, when's the next round starting?"<d><scn>black<msg>Br "Soon enough. Do you feel it? Do you feel the buzz kicking in already?"<|endoftext|><d><scn>facin2<msg>An "Yeah, I'm very sorry we can
't accommodate your request right now, your highness."<|endoftext|>fur", but "<|endoftext|><scn>o2<msg>Br "My point with all this is: We're all a very tight knit group at the police station. Me, Sebastian, Maverick, Naomi - the bond we form on the job is how we know to look out for each other."<|endoftext|><p><msg>c "Let's start with the stone. Ideally, we want one that has a big surface area, but is as flat as possible."<d><scn>beach<msg>Ad "Got it."<|endoftext|><d
[0] set freeze_part_layers: True (freezing 150 out of 160 layers.)


Step,Training Loss
52,3.118700
104,2.229700
156,2.092700
208,1.999300
260,1.892900
312,1.798800
364,1.705600
416,1.694300
468,1.572700
520,1.497600


In [ ]:
fig, axs = plt.subplots(2)
fig.suptitle('Learning rate and loss')
axs[0].plot(train_results['learning_rate_history'])
axs[1].plot(train_results['loss_history'])

# Testing

We created a few past (for context) + present prompts (player input) and see the different reactions. This way, we can test the models across different iterations.

In [ ]:
train_results['model'].eval()
model_manager = ModelManager(model=train_results['model'], tokenizer=train_results['tokenizer'])

In [ ]:
prompts = [
    ('<p><msg>c "Hey Remy!"<d><msg>Ry "Hey!"', "How are you?"),
    ('<p><msg>c "I was with Lorem today."<d><msg>Ad "Very nice."', "What do you think of Lorem?"),
    ('<p><msg>m "In Tatsu park, Adine and I sat down."', "Oh my god, Adine. What is this?"),
    ('<p><msg>m "I sat down on a chair in Anna\'s lab."', "What will we do here?"),
]

for (past, prompt) in prompts:
    reply = model_manager.say(past, prompt)
    print(f"Prompt: {prompt}\nReply: {reply}\n\n")

# Sampling test

Which combination is the best?

In [ ]:
for i in range(100):
    torch.manual_seed(80085)
    top_k = random.randint(0, 100)
    top_p = round(random.uniform(0, 1), 2)
    for (past, prompt) in prompts:
        reply = model_manager.say(past, prompt, top_k = top_k, top_p = top_p)
        print(f"[Test {i + 1} top_k: {top_k}, top_p: {top_p}] -> Prompt: {prompt}\nReply: {reply}\n")
    print("-------------")

In [ ]:
print("What to say?")
print(generate_dragon_reply("", input()))